In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
#import gw biweekly 2021 and gw biweekly full

gw_full = pd.read_csv('groundwater_biweekly_full.csv')
gw_2021 = pd.read_csv('groundwater_biweekly_2021_field.csv')
well_dimensions = pd.read_csv('well_dimensions.csv')


In [3]:
#copy gw_2021 dataframe but with new blank column for ground to water
gw_2021_updated = gw_2021
gw_2021_updated['ground_to_water_cm'] = 0

gw_2021_updated['timestamp'] = pd.to_datetime(gw_2021_updated['timestamp'])
gw_2021_updated["date"] = [d.date() for d in gw_2021_updated["timestamp"]]
gw_2021_updated

,well_id,timestamp,welltop_to_water_cm,water_level_1_cm,water_level_2_cm,water_binary,logger?,soil moisture,notes,ground_to_water_cm,date
0,EHR-1,2021-05-20 07:38:00,100.2792,100.2792,100.2792,1,False,NaN,NaN,0,2021-05-20
1,EER-1,2021-05-20 07:40:00,107.1880,106.9848,107.2896,1,False,NaN,NaN,0,2021-05-20
2,EWR-1,2021-05-20 07:42:00,107.5944,107.5944,107.5944,1,False,NaN,NaN,0,2021-05-20
3,EFF-XA1N,2021-05-20 07:52:00,136.6520,136.5504,136.5504,1,False,NaN,NaN,0,2021-05-20
4,EFF-XA2N,2021-05-20 07:50:00,54.6608,54.5592,54.5592,1,False,NaN,NaN,0,2021-05-20
...,...,...,...,...,...,...,...,...,...,...,...
463,KHF-1,2021-11-14 08:25:00,42.0624,42.0624,42.0624,1,False,NaN,NaN,0,2021-11-14
464,KWF-1,2021-11-14 08:33:00,70.7136,70.7136,70.7136,1,False,NaN,"hole needs backfill, -2 cm from ground surface",0,2021-11-14
465,KWR-1,2021-11-14 08:38:00,85.9536,85.9536,85.9536,1,False,NaN,NaN,0,2021-11-14
466,EET-XB4S,2021-11-14 10:06:00,84.1248,84.1248,84.1248,1,False,NaN,NaN,0,2021-11-14


In [4]:
well_dimensions['timestamp'] = pd.to_datetime(well_dimensions['timestamp'])
well_dimensions['date'] = [d.date() for d in well_dimensions['timestamp']]
well_dimensions

,well_id,timestamp,welltop_to_ground,total_well_length,ground_elevation,welltop_elevation,meter_id,notes,Unnamed: 8,date
0,EHR-1,2018-05-31 08:30:00,53.9750,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31
1,EHR-1,2018-05-31 08:30:00,53.9750,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31
2,EER-1,2018-05-31 08:40:00,78.4352,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31
3,EER-1,2018-05-31 08:40:00,77.4700,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31
4,EER-1,2018-05-31 08:40:00,78.1050,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31
...,...,...,...,...,...,...,...,...,...,...
515,KHT-XE6S,2021-07-19 00:00:00,50.9250,NaN,NaN,NaN,NaN,new well,NaN,2021-07-19
516,EHF-XA3S,2021-07-19 00:00:00,59.6500,NaN,NaN,NaN,NaN,new well,NaN,2021-07-19
517,EWT-1,2021-07-19 00:00:00,64.5000,NaN,NaN,NaN,NaN,new well,NaN,2021-07-19
518,KWR-1,2021-06-20 00:00:00,71.3400,NaN,NaN,NaN,NaN,new well,NaN,2021-06-20


In [5]:

differences = []
well_ids = []
dates = []
for index1, row1 in gw_2021_updated.iterrows():      #iterate through gw_2021 rows
    well_id_gw = row1['well_id']
    date_gw = row1['date']
    well_dim_filtered = well_dimensions[well_dimensions['well_id'] == well_id_gw].sort_values(by='date')
    well_dim_date_filtered = well_dim_filtered[well_dim_filtered['date'] <= date_gw]
    if well_dim_date_filtered.empty:
        #if well_dim_filtered.empty:
        differences.append(np.nan)
#         else:
#             well_ids.append(well_id_gw)
#             dates.append(date_gw)
#             row2 = well_dim_filtered.iloc[0]
#             differences.append(row1['welltop_to_water_cm'] - row2['welltop_to_ground'])
    else:
        row2 = well_dim_date_filtered.iloc[-1]
        differences.append(row1['welltop_to_water_cm'] - row2['welltop_to_ground'])
        
gw_2021_updated['ground_to_water_cm'] = differences
print(well_ids)
print(dates)
gw_2021_updated



[]
[]


,well_id,timestamp,welltop_to_water_cm,water_level_1_cm,water_level_2_cm,water_binary,logger?,soil moisture,notes,ground_to_water_cm,date
0,EHR-1,2021-05-20 07:38:00,100.2792,100.2792,100.2792,1,False,NaN,NaN,44.6792,2021-05-20
1,EER-1,2021-05-20 07:40:00,107.1880,106.9848,107.2896,1,False,NaN,NaN,28.6880,2021-05-20
2,EWR-1,2021-05-20 07:42:00,107.5944,107.5944,107.5944,1,False,NaN,NaN,16.2944,2021-05-20
3,EFF-XA1N,2021-05-20 07:52:00,136.6520,136.5504,136.5504,1,False,NaN,NaN,NaN,2021-05-20
4,EFF-XA2N,2021-05-20 07:50:00,54.6608,54.5592,54.5592,1,False,NaN,NaN,NaN,2021-05-20
...,...,...,...,...,...,...,...,...,...,...,...
463,KHF-1,2021-11-14 08:25:00,42.0624,42.0624,42.0624,1,False,NaN,NaN,42.0624,2021-11-14
464,KWF-1,2021-11-14 08:33:00,70.7136,70.7136,70.7136,1,False,NaN,"hole needs backfill, -2 cm from ground surface",NaN,2021-11-14
465,KWR-1,2021-11-14 08:38:00,85.9536,85.9536,85.9536,1,False,NaN,NaN,14.6136,2021-11-14
466,EET-XB4S,2021-11-14 10:06:00,84.1248,84.1248,84.1248,1,False,NaN,NaN,NaN,2021-11-14


In [7]:
#save csv
gw_2021_updated.to_csv(r'\Desktop\sagehen_meadows\data\field_observations\groundwater\biweekly_manual\groundwater_biweekly_full_2018_2021.csv', index = False)

